In [1]:
# Carrega as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# 1- Café

In [2]:
# Modifica o nome das colunas ligando elas ao nome da tabela
i=e_c_m_r_pa_clean=pd.read_csv(   '~/git/comex_sh_mun/temp/e_c_m_r_pa_clean.csv')
i.head(1)

,CO_ANO,CO_MES,SH4_c,CO_PAIS_i_c,NO_MUN_e_c,KG_c,VL_c
0,1997,3,901,999,OUTROS,300000.0,514554


In [3]:
# Cria a coluna 'date', que servirá para o controle das bordas da matriz esparsa
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1')
i.sort_values('date')

,CO_ANO,CO_MES,SH4_c,CO_PAIS_i_c,NO_MUN_e_c,KG_c,VL_c,date
209,1997,1,901,999,OUTROS,38400.0,65280,1997-01-01
303,1997,1,901,999,OUTROS,153600.0,177578,1997-01-01
99,1997,1,901,999,OUTROS,115200.0,133379,1997-01-01
106,1997,1,901,999,OUTROS,76800.0,79493,1997-01-01
264,1997,1,901,999,OUTROS,280800.0,361378,1997-01-01
...,...,...,...,...,...,...,...,...
13221,2024,1,901,999,OUTROS,448775.0,1638643,2024-01-01
13222,2024,1,901,999,OUTROS,40800.0,110009,2024-01-01
13223,2024,1,901,999,OUTROS,52800.0,172152,2024-01-01
13225,2024,1,901,999,OUTROS,556075.0,1657498,2024-01-01


Os procedimeno a seguir tem objetivo de, ao transformar a base de dados em matriz esparsa, não permitir que as bordas ocupem datas não cobertas pela arquivo original

In [4]:
# Identifica as bordas
i_date_min=i.date.min()
print(i_date_min)
i_date_max=i.date.max()
print(i_date_max)

i_ano_min=i.CO_ANO.min()
i_ano_max=i.CO_ANO.max()

# Cria df com as datas completas
q_date=pd.DataFrame({'CO_ANO':np.arange(i_ano_min,i_ano_max+1),'key':0})\
.merge(pd.DataFrame({'CO_MES':np.arange(1,13),'key':0}),how='outer').drop(columns='key')

q_date['date']=pd.to_datetime(q_date.CO_ANO.astype(str)+'/'+q_date.CO_MES.astype(str)+'/1')

q_date=q_date[(q_date.date>=i_date_min)&(q_date.date<=i_date_max)]
q_date=q_date.date
q_date

1997-01-01 00:00:00
2024-01-01 00:00:00


0     1997-01-01
1     1997-02-01
2     1997-03-01
3     1997-04-01
4     1997-05-01
         ...    
320   2023-09-01
321   2023-10-01
322   2023-11-01
323   2023-12-01
324   2024-01-01
Name: date, Length: 325, dtype: datetime64[ns]

In [5]:
# Mofifica a base de dados para o tipo 'categoria'
i[i.columns[1:5]]=i[i.columns[1:5]].astype('category')
i.head(1)

,CO_ANO,CO_MES,SH4_c,CO_PAIS_i_c,NO_MUN_e_c,KG_c,VL_c,date
0,1997,3,901,999,OUTROS,300000.0,514554,1997-03-01


In [6]:
# Cria  a matriz esparsa da base de dados
i_sprx=i.groupby(by=i.columns[:5].tolist())[i.columns[5:-1].tolist()].sum().reset_index()
#i_sprx=i_sprx.set_index('CO_ANO')
i_sprx['date']=pd.to_datetime(i_sprx.CO_ANO.astype(str)+'/'+i_sprx.CO_MES.astype(str)+'/1')
i_s=i_sprx

In [7]:
i_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CO_ANO       336 non-null    int64         
 1   CO_MES       336 non-null    category      
 2   SH4_c        336 non-null    category      
 3   CO_PAIS_i_c  336 non-null    category      
 4   NO_MUN_e_c   336 non-null    category      
 5   KG_c         336 non-null    float64       
 6   VL_c         336 non-null    int64         
 7   date         336 non-null    datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(1), int64(2)
memory usage: 12.7 KB


In [8]:
# Limita a matriz esparsa ao quadro de datas
i_s=i_s.merge(q_date)

In [9]:
i_s=i_s.sort_values('date').drop(columns='date')
i_s

,CO_ANO,CO_MES,SH4_c,CO_PAIS_i_c,NO_MUN_e_c,KG_c,VL_c
0,1997,1,901,999,OUTROS,6.507000e+06,7121382
1,1997,2,901,999,OUTROS,6.357480e+06,8277927
2,1997,3,901,999,OUTROS,7.063920e+06,10768024
3,1997,4,901,999,OUTROS,4.025400e+06,6416305
4,1997,5,901,999,OUTROS,6.816000e+06,12977961
...,...,...,...,...,...,...,...
320,2023,9,901,999,OUTROS,9.496183e+06,31254986
321,2023,10,901,999,OUTROS,1.402960e+07,44305561
322,2023,11,901,999,OUTROS,1.388889e+07,43663488
323,2023,12,901,999,OUTROS,1.665014e+07,53573926


In [10]:
i_s[['CO_MES','NO_MUN_e_c','SH4_c','CO_PAIS_i_c']]=i_s[['CO_MES','NO_MUN_e_c','SH4_c','CO_PAIS_i_c']].astype(str)

In [11]:
i_s['x']=i_s['CO_MES'].str.cat(i_s[['NO_MUN_e_c','SH4_c','CO_PAIS_i_c']],sep='-')

In [12]:
i_s.head(1)

,CO_ANO,CO_MES,SH4_c,CO_PAIS_i_c,NO_MUN_e_c,KG_c,VL_c,x
0,1997,1,901,999,OUTROS,6507000.0,7121382,1-OUTROS-901-999


In [13]:
k=i_s.groupby('CO_ANO').KG_c.sum().reset_index().merge(i_s.pivot(index='CO_ANO',columns=['x'],values='KG_c').reset_index())
k.CO_ANO=k.CO_ANO+1
k

,CO_ANO,KG_c,1-OUTROS-901-999,10-OUTROS-901-999,11-OUTROS-901-999,12-OUTROS-901-999,2-OUTROS-901-999,3-OUTROS-901-999,4-OUTROS-901-999,5-OUTROS-901-999,6-OUTROS-901-999,7-OUTROS-901-999,8-OUTROS-901-999,9-OUTROS-901-999
0,1998,6.278062e+07,6.507000e+06,5.398800e+06,3.919200e+06,4.758000e+06,6.357480e+06,7.063920e+06,4.025400e+06,6.816000e+06,2.614200e+06,6.649180e+06,3.779640e+06,4.891800e+06
1,1999,8.478444e+07,4.993800e+06,8.786880e+06,8.032680e+06,8.448000e+06,4.938000e+06,5.660160e+06,5.202600e+06,5.650800e+06,5.471160e+06,8.686800e+06,1.016796e+07,8.745600e+06
2,2000,7.897412e+07,5.874118e+06,9.147647e+06,5.920000e+06,3.779412e+06,6.574118e+06,8.418824e+06,8.201765e+06,6.443529e+06,6.024118e+06,4.450000e+06,6.982941e+06,7.157647e+06
3,2001,6.914785e+07,5.745057e+06,6.247925e+06,5.899245e+06,5.276604e+06,7.058491e+06,4.206453e+06,5.708604e+06,6.540566e+06,5.889057e+06,3.900000e+06,6.327170e+06,6.348679e+06
4,2002,5.903253e+07,6.186000e+06,4.817296e+06,5.654727e+06,3.628145e+06,4.932764e+06,5.077091e+06,3.817091e+06,4.687418e+06,4.760727e+06,4.664727e+06,6.111273e+06,4.695273e+06
5,2003,6.166527e+07,4.478571e+06,7.302712e+06,6.141020e+06,5.421273e+06,3.403393e+06,3.653143e+06,4.449643e+06,2.157589e+06,3.670393e+06,7.126193e+06,5.982412e+06,7.878932e+06
6,2004,6.713127e+07,7.489768e+06,6.422842e+06,6.157895e+06,6.496632e+06,6.173644e+06,3.719686e+06,5.354807e+06,3.829549e+06,4.648067e+06,4.486663e+06,5.438439e+06,6.913281e+06
7,2005,8.696588e+07,5.501695e+06,9.288198e+06,1.074010e+07,7.289075e+06,4.943668e+06,6.346169e+06,5.201310e+06,6.796271e+06,6.786610e+06,6.571932e+06,7.598142e+06,9.902705e+06
8,2006,1.020955e+08,9.207811e+06,7.753115e+06,8.413095e+06,8.568426e+06,7.803036e+06,1.110927e+07,9.930689e+06,7.844382e+06,8.554661e+06,7.285615e+06,9.275705e+06,6.349651e+06
9,2007,1.053098e+08,7.429257e+06,9.413440e+06,1.033419e+07,9.385373e+06,1.025444e+07,7.738010e+06,6.375070e+06,6.799230e+06,7.709500e+06,8.015060e+06,9.811444e+06,1.204477e+07


In [14]:
k.to_csv('~/git/comex_sh_mun/temp/e_c_m_r_pa_clean_wide.csv',index=False)
k

,CO_ANO,KG_c,1-OUTROS-901-999,10-OUTROS-901-999,11-OUTROS-901-999,12-OUTROS-901-999,2-OUTROS-901-999,3-OUTROS-901-999,4-OUTROS-901-999,5-OUTROS-901-999,6-OUTROS-901-999,7-OUTROS-901-999,8-OUTROS-901-999,9-OUTROS-901-999
0,1998,6.278062e+07,6.507000e+06,5.398800e+06,3.919200e+06,4.758000e+06,6.357480e+06,7.063920e+06,4.025400e+06,6.816000e+06,2.614200e+06,6.649180e+06,3.779640e+06,4.891800e+06
1,1999,8.478444e+07,4.993800e+06,8.786880e+06,8.032680e+06,8.448000e+06,4.938000e+06,5.660160e+06,5.202600e+06,5.650800e+06,5.471160e+06,8.686800e+06,1.016796e+07,8.745600e+06
2,2000,7.897412e+07,5.874118e+06,9.147647e+06,5.920000e+06,3.779412e+06,6.574118e+06,8.418824e+06,8.201765e+06,6.443529e+06,6.024118e+06,4.450000e+06,6.982941e+06,7.157647e+06
3,2001,6.914785e+07,5.745057e+06,6.247925e+06,5.899245e+06,5.276604e+06,7.058491e+06,4.206453e+06,5.708604e+06,6.540566e+06,5.889057e+06,3.900000e+06,6.327170e+06,6.348679e+06
4,2002,5.903253e+07,6.186000e+06,4.817296e+06,5.654727e+06,3.628145e+06,4.932764e+06,5.077091e+06,3.817091e+06,4.687418e+06,4.760727e+06,4.664727e+06,6.111273e+06,4.695273e+06
5,2003,6.166527e+07,4.478571e+06,7.302712e+06,6.141020e+06,5.421273e+06,3.403393e+06,3.653143e+06,4.449643e+06,2.157589e+06,3.670393e+06,7.126193e+06,5.982412e+06,7.878932e+06
6,2004,6.713127e+07,7.489768e+06,6.422842e+06,6.157895e+06,6.496632e+06,6.173644e+06,3.719686e+06,5.354807e+06,3.829549e+06,4.648067e+06,4.486663e+06,5.438439e+06,6.913281e+06
7,2005,8.696588e+07,5.501695e+06,9.288198e+06,1.074010e+07,7.289075e+06,4.943668e+06,6.346169e+06,5.201310e+06,6.796271e+06,6.786610e+06,6.571932e+06,7.598142e+06,9.902705e+06
8,2006,1.020955e+08,9.207811e+06,7.753115e+06,8.413095e+06,8.568426e+06,7.803036e+06,1.110927e+07,9.930689e+06,7.844382e+06,8.554661e+06,7.285615e+06,9.275705e+06,6.349651e+06
9,2007,1.053098e+08,7.429257e+06,9.413440e+06,1.033419e+07,9.385373e+06,1.025444e+07,7.738010e+06,6.375070e+06,6.799230e+06,7.709500e+06,8.015060e+06,9.811444e+06,1.204477e+07
